# Assignment 2: Language Modelling in Hangman

Student Name: Matthias Bachfischer

Student ID: 1133751

## General info

<b>Due date</b>: Tuesday, 6 April 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [4]:
if interactive:
    hangman('plague', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import brown
import numpy as np

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###

# Compute # of unique word types
import re
from nltk.corpus import brown

# Create set of unique word types
word_types = set()


# words from brown corpus
brown_words = brown.words()

# Select words entirely compromised of alphabetic characters
r = re.compile("^[a-zA-Z]+$")

brown_words_filtered = list(filter(r.match, brown_words))

# lower-case all words
brown_words_lower = [word.lower() for word in brown_words_filtered]

# obtain word types
for word in brown_words_lower:
    word_types.add(word)
    
word_types = list(word_types)

# randomly shuffle collection of word types
np.random.shuffle(word_types)

test_set = word_types[0:1000]
training_set = word_types[1000:len(word_types)]

print("Number of word types in total =", len(word_types))

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in total = 40234
Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [6]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [7]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [8]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [9]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    import random
    import string
    
    guess = random.choice(string.ascii_letters).lower()
    while guess in guessed:
        guess = random.choice(string.ascii_letters).lower()
    return guess

    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = unclouded
Number of mistakes made by the random guesser = 18

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.539


<b>For your testing:</b>

In [10]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [11]:
unigram_counts = None

###
# Your answer BEGINS HERE
###
from collections import Counter, OrderedDict


# find frequeny of each character in training set
def get_unigram_counts(words):
    unigram_counts = Counter()

    # collect initial unigram statistics
    for word in words: 
        counter = {char:word.count(char) for char in set(word)}
        unigram_counts += counter

    return unigram_counts
    
unigram_counts = get_unigram_counts(training_set)

###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    
    # Dict that holds probabilities for each character a-z
    probabilities_per_character = {}
    
    total_counts = float(sum(unigram_counts.values()))

    # update probabilities for each character
    for character in unigram_counts:
        if character not in probabilities_per_character:
            probabilities_per_character[character] = []
        probabilities_per_character[character].append(unigram_counts[character] / total_counts)
    
    
    probabilities_ordered = OrderedDict(sorted(probabilities_per_character.items(), key=lambda t: t[1], reverse=True))
    guesses = list(probabilities_ordered.keys())
    
    # deal with missing characters (i.e. make sure we try every character in the alphabet )
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    if not all(char in guesses for char in alphabet):
        missing_chars = list(set(alphabet) - set(guesses))
        # add missing characters to list of guesses (in case there are any missing values)
        guesses.extend(missing_chars)
    
    position = 0
    guess = guesses[position]
    while guess in guessed:
        position += 1
        guess = guesses[position]
    return guess

    ###
    # Your answer ENDS HERE
    ###


result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.407


<b>For your testing:</b>

In [12]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [13]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###


# find frequeny of each character in training set
def get_unigram_counts_by_length(words):
    unigram_counts_by_length = {}

    # collect unigram statistics conditioned by length
    for word in words: 
        word_length = len(word)
        
        if word_length not in unigram_counts_by_length:
            unigram_counts_by_length[word_length] = Counter()
            
        counter = {char:word.count(char) for char in set(word)}
        unigram_counts_by_length[word_length] += counter
    
    return unigram_counts_by_length
    
unigram_counts_by_length = get_unigram_counts_by_length(training_set)

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    target_string_length = len(mask)
    
    # Dict that holds probabilities for each character a-z
    probabilities_per_character = {}
    
    if target_string_length in unigram_counts_by_length:
        target_unigram_model = unigram_counts_by_length[target_string_length]
    else:
        target_unigram_model = unigram_counts
    
    total_counts = float(sum(target_unigram_model.values()))

    # update probabilities for each character
    for character in target_unigram_model:
        if character not in probabilities_per_character:
            probabilities_per_character[character] = []
        probabilities_per_character[character].append(target_unigram_model[character] / total_counts)
        
    probabilities_ordered = OrderedDict(sorted(probabilities_per_character.items(), key=lambda t: t[1], reverse=True))
    guesses = list(probabilities_ordered.keys())
    
    # deal with missing characters (i.e. make sure we try every character in the alphabet )
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    if not all(char in guesses for char in alphabet):
        missing_chars = list(set(alphabet) - set(guesses))
        # add missing characters to list of guesses (in case there are any missing values)
        guesses.extend(missing_chars)
    
    position = 0
    guess = guesses[position]
    while guess in guessed:
        position += 1
        guess = guesses[position]
    return guess
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.342


<b>For your testing:</b>

In [14]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [15]:
bigram_counts = None

###
# Your answer BEGINS HERE
###
from collections import defaultdict
from collections import Counter

def convert_word(word):
    return ["<s>"] + list(word)

def get_bigram_counts(words):
    
    bigram_counts = defaultdict(Counter)

    for word in words:
        
        # obtain bigram counts
        word = convert_word(word)
        # generate a list of bigrams
        bigram_list = zip(word[:-1], word[1:])
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
            
    return bigram_counts

bigram_counts = get_bigram_counts(training_set)
###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts=unigram_counts): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    
    target_string_length = len(mask)
        
    # Dict that holds probabilities for each character a-z
    probabilities_per_character = {}
    
    for i in range(target_string_length): 
        # if character at current position has not been identified yet
        if(mask[i] == "_"):
            # if we are not at the beginning of the word
            if i >= 1 and mask[i-1] != "_":
                prev_counts = bigram_counts[mask[i-1]]
            # if we are at the beginning of the word
            elif i == 0:
                prev_counts = bigram_counts["<s>"]
            else:
                prev_counts = unigram_counts
            
            total_counts = float(sum(prev_counts.values()))

            # update probabilities for each character
            for character in prev_counts:
                if character not in probabilities_per_character:
                    probabilities_per_character[character] = []

                probabilities_per_character[character].append(prev_counts[character] / total_counts)

    # Get the sum of probabilties for each character
    for char in probabilities_per_character:
        probabilities_per_character[char] = sum(probabilities_per_character[char])
    
    # Order characters in descending order by their probabilities
    probabilities_ordered = OrderedDict(sorted(probabilities_per_character.items(), key=lambda t: t[1], reverse=True))
    guesses = list(probabilities_ordered.keys())
    
    # deal with missing characters (i.e. make sure we try every character in the alphabet )
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    if not all(char in guesses for char in alphabet):
        missing_chars = list(set(alphabet) - set(guesses))
        # add missing characters to list of guesses (in case there are any missing values)
        guesses.extend(missing_chars)
    
    position = 0
    guess = guesses[position]
    while guess in guessed:
        position += 1
        guess = guesses[position]
    return guess
    ###
    # Your answer ENDS HERE
    ###



result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.867


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 8.0
* 1.0 mark if 8.0 <= x < 8.5
* 0.5 mark if 8.5 <= x < 8.8
* 0.0 mark if x >= 8.8

Note: when testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [16]:
###
# Your answer BEGINS HERE
###

from nltk.util import ngrams

# Randomly split data into training / test sets using different seed values
def random_train_test_split(seed):
    np.random.seed(seed)

    # training_set stores the rest word types for training
    training_set = []
    # test_set stores 1000 word types for testing
    test_set = []

    # randomly shuffle collection of word types
    np.random.shuffle(word_types)

    test_set = word_types[0:1000]
    training_set = word_types[1000:len(word_types)]
    return (test_set, training_set)

test_set, training_set = random_train_test_split(seed=100)

trigram_counts = None
fourgram_counts = None

def get_ngram_counts(words):
    
    trigram_counts = defaultdict(lambda: defaultdict(Counter))
    fourgram_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))

    for word in words:
        
        # add prefix to word
        word = convert_word(word)
        
        # generate list of trigrams
        trigram_list = list(ngrams(word, 3))
        # iterate over trigrams
        for trigram in trigram_list:
            first, second, third = trigram
            trigram_counts[first][second][third] += 1
            
        # generate list of fourgrams
        fougram_list = list(ngrams(word, 4))
        # iterate over fourgrams
        for fourgram in fougram_list:
            first, second, third, fourth = fourgram
            fourgram_counts[first][second][third][fourth] += 1
            
    return (trigram_counts, fourgram_counts)

trigram_counts, fourgram_counts = get_ngram_counts(training_set)

def my_amazing_ai_guesser(mask, guessed, fourgram_counts = fourgram_counts, trigram_counts = trigram_counts, bigram_counts=bigram_counts, unigram_counts=unigram_counts, unigram_counts_by_length=unigram_counts_by_length, k = 0.8):
    
    target_string_length = len(mask)
    
    # Dict that holds probabilities for each character a-z
    probabilities_per_character = {}
    
    for i in range(target_string_length): 
        # if character at current position has not been identified yet
        if(mask[i] == "_"):
            
            # if previous three characters are known
            if i >= 3 and mask[i-3] != "_" and mask[i-2] != "_" and mask[i-1] != "_":
                prev_counts = fourgram_counts[mask[i-3]][mask[i-2]][mask[i-1]]
            # if previous two characters are known and we are at the beginning of the sentence
            elif i >= 2 and (i-2) == 0 and mask[i-2] != "_" and mask[i-1] != "_":
                prev_counts = fourgram_counts["<s>"][mask[i-2]][mask[i-1]]
            # if previous two characters are known and we are not at the beginning of sentence
            elif i >= 2 and mask[i-2] != "_" and mask[i-1] != "_":     
                prev_counts = trigram_counts[mask[i-2]][mask[i-1]]
            # if previous one character is known and we are at the beginning of the sentence
            elif i >= 1 and (i-1) == 0 and mask[i-1] != "_":
                prev_counts = trigram_counts["<s>"][mask[i-1]]
            # if previous one character is known
            elif i >= 1 and mask[i-1] != "_":
                prev_counts = bigram_counts[mask[i-1]]
            # if we are at the beginning of the sentence
            elif i == 0:
                prev_counts = bigram_counts["<s>"]
            # if previous character is unknow (i.e. previous character is "_"), then we use unigrams conditioned by length
            elif target_string_length in unigram_counts_by_length:
                prev_counts = unigram_counts_by_length[target_string_length]
            else:
                prev_counts = unigram_counts
                         
            total_counts = float(sum(prev_counts.values()))

            # update probabilities for each character
            for character in prev_counts:
                if character not in probabilities_per_character:
                    probabilities_per_character[character] = []

                probabilities_per_character[character].append((prev_counts[character] + k) / (total_counts + k*len(unigram_counts)))

    # Get the sum of probabilties for each character
    for char in probabilities_per_character:
        probabilities_per_character[char] = sum(probabilities_per_character[char])
        
    # Order characters in descending order by their probabilities
    probabilities_ordered = OrderedDict(sorted(probabilities_per_character.items(), key=lambda t: t[1], reverse=True))
    guesses = list(probabilities_ordered.keys())
    
    # deal with missing characters (i.e. make sure we try every character in the alphabet )
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    if not all(char in guesses for char in alphabet):
        missing_chars = list(set(alphabet) - set(guesses))
        # add missing characters to list of guesses (in case there are any missing values)
        guesses.extend(missing_chars)

    position = 0
    guess = guesses[position]
    while guess in guessed:
        position += 1
        guess = guesses[position]
    return guess

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.37


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

In my approach towards developing a more efficient AI approach for playing hangman, I have mainly focused on extending the previous bigram language model from question 5 and incorporated more advanced prediction strategies.

The prediction strategies that are used in *my_amazing_ai_guesser* involves:
* Usage of trigrams and fourgrams (in addition to the  bigrams and unigrams that were previously used in question 5). This has helped to significantly boost prediction performance!
* Usage of different unigram models depending on the length of the word to be guessed (i.e. usage of the *unigram_counts_by_length* frequencies obtained in question 4).
* Usage of add-k smoothing with *k=0.8* to be able to better deal with unseen words

The model performance has been evaluated by performing multiple testing rounds with different train / test splits via the *random_train_test_split* function (using different seed values).

**Overall, by incorporating the strategies mentioned above I was able to significantly boost model performance - the average number of incorrect guesses has averaged around 7.2 during my experiments.**

##### Your answer ENDS HERE